In [4]:
import netCDF4
import pandas as pd
import numpy as np
import xarray as xr
from db_queries import get_outputs, get_ids

reading nc files: https://gist.github.com/jswhit/43dc116595ed66a92804
make sure request large memory like 50GB

In [5]:
%cd /ihme/costeffectiveness/vivarium_csu_cancer
!ls -halt

/ihme/costeffectiveness/vivarium_csu_cancer
total 1.9G
-rw-r--r--  1 nicoly   Domain Users  68K Jun 18 22:42 c429_prev_blendedlocation_forecast.csv
drwxrwx---  2 plinds   Domain Users 3.5K Jun 18 22:42 .
-rw-r--r--  1 nicoly   Domain Users  63K Jun 18 22:24 c429_csmr_blendedlocation_forecast.csv
-rw-r--r--  1 nicoly   Domain Users  59K Jun 18 21:28 acmr_blendedlocation_forecast.csv
-rwxrwx---  1 plinds   Domain Users 437M Jun  4 12:34 429_ets_prevalence_beta_8_phi_89.nc
drwxrwxrwx 17 emumford Domain Users 9.0K Jun  3 17:46 ..
-rwxrwx---  1 plinds   Domain Users 591M Jun  2 18:00 294_ets_mortality_beta_8_phi_89.nc
-rwxrwx---  1 plinds   Domain Users 437M May 13 16:58 429_ets_direct_incidence.nc
-rwxrwx---  1 plinds   Domain Users 437M May 13 16:58 429_ets_mortality.nc


In [6]:
#model location and id lists

model_location_ids_list=[496,501,502,506,517]
model_years_list=list(range(2020, 2040))
model_agegroup_list=list(range(8,21)) + [30,31,32,235]
print(model_agegroup_list)
type(model_agegroup_list)

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 30, 31, 32, 235]


list

In [12]:
province_weights = {517:0.18, 506:0.28, 496:0.22, 502:0.16, 501:0.16}

age = {8:'15 to 19',
       9:'20 to 24',
       10:'25 to 29',
       11:'30 to 34',
       12:'35 to 39',
       13:'40 to 44',
       14:'45 to 49',
       15:'50 to 54',
       16:'55 to 59',
       17:'60 to 64',
       18:'65 to 69',
       19:'70 to 74',
       20:'75 to 79',
       30:'80 to 84',
       31:'85 to 89',
       32:'90 to 94',
       235:'95 plus'}

sex = {1:'male',2:'female'}

# ACMR

In [172]:
#load .nc data

f_acmr = netCDF4.Dataset('294_ets_mortality_beta_8_phi_89.nc', 'r')
print(f_acmr)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): age_group_id(23), sex_id(2), location_id(33), year_id(51), draw(1000)
    variables(dimensions): int64 age_group_id(age_group_id), int64 sex_id(sex_id), int64 location_id(location_id), int64 year_id(year_id), <class 'str'> draw(draw), float64 mr(age_group_id,sex_id,location_id,year_id,draw)
    groups: 


In [173]:
#details of 'mr' column

acmr_mr=f_acmr.variables['mr']
print(acmr_mr)

<class 'netCDF4._netCDF4.Variable'>
float64 mr(age_group_id, sex_id, location_id, year_id, draw)
    _FillValue: nan
unlimited dimensions: 
current shape = (23, 2, 33, 51, 1000)
filling on


In [174]:
acmr_mr_array = acmr_mr[:]
#print(acmr_mr_array)

# Convert .nc file to pandas dataframe: acmr_all

In [66]:
#1 convert .nc file to pandas dataframe 

ds_acmr = xr.open_dataset('294_ets_mortality_beta_8_phi_89.nc') #all-cause mortality file
acmr_all = ds_acmr.to_dataframe().rename(columns={'mr':'log_mr'}).reset_index()  
acmr_all.head()

,age_group_id,draw,location_id,sex_id,year_id,log_mr
0,2,0,354,1,1990,-1.852232
1,2,0,354,1,1991,-1.935975
2,2,0,354,1,1992,-1.989802
3,2,0,354,1,1993,-2.198598
4,2,0,354,1,1994,-2.292911


In [67]:
#2 adding a column that is the e(log_mr) to get the mr

acmr_all['mr']=np.exp(acmr_all.log_mr)
acmr_all.head()

,age_group_id,draw,location_id,sex_id,year_id,log_mr,mr
0,2,0,354,1,1990,-1.852232,0.156887
1,2,0,354,1,1991,-1.935975,0.144283
2,2,0,354,1,1992,-1.989802,0.136723
3,2,0,354,1,1993,-2.198598,0.110959
4,2,0,354,1,1994,-2.292911,0.100972


In [68]:
acmr_all['mr_100Kpy']=acmr_all.mr*100000
acmr_all.head()

,age_group_id,draw,location_id,sex_id,year_id,log_mr,mr,mr_100Kpy
0,2,0,354,1,1990,-1.852232,0.156887,15688.662762
1,2,0,354,1,1991,-1.935975,0.144283,14428.346328
2,2,0,354,1,1992,-1.989802,0.136723,13672.252803
3,2,0,354,1,1993,-2.198598,0.110959,11095.859481
4,2,0,354,1,1994,-2.292911,0.100972,10097.212142


In [69]:
acmr_all.loc[acmr_all.year_id == 2017, 'mr_100Kpy'].describe()

count    1.518000e+06
mean     4.992530e+03
std      9.450920e+03
min      5.214105e+00
25%      6.574647e+01
50%      4.571590e+02
75%      4.233560e+03
max      7.260642e+04
Name: mr_100Kpy, dtype: float64

In [70]:
acmr_all.loc[(acmr_all.year_id == 2017) & (acmr_all.age_group_id == 8), 'mr_100Kpy'].describe()

count    66000.000000
mean        32.910264
std         20.571047
min          5.675230
25%         17.519875
50%         26.384080
75%         43.658114
max        174.351538
Name: mr_100Kpy, dtype: float64

In [71]:
#age-specific all cause mortality rate

acmr_all.loc[acmr_all.year_id == 2017].groupby(['year_id','age_group_id','sex_id']).mr_100Kpy.describe(percentiles=[.025, 0.975]).filter(['mean', '2.5%', '97.5%'])

##-this is all locations, all ages in forecast/We eye-balled it to GBD 2017 and the age-specific acmrs are similar. 


mean          2.5%         97.5%
year_id age_group_id sex_id                                          
2017    2            1       21055.214640   2757.309557  62926.621479
                     2       17910.946343   2595.390726  49292.051497
        3            1        1997.736591    223.514003   5901.675111
                     2        1867.824918    271.138280   4974.922203
        4            1         607.565671     52.085606   2143.881667
                     2         457.579234     39.821313   1474.584456
        5            1          63.465150      9.060806    197.727296
                     2          55.109256     11.162563    154.418270
        6            1          43.142889      7.049343     90.944699
                     2          29.523143      6.127632     60.286579
        7            1          43.628658      9.178479    103.765602
                     2          25.018868      7.309599     47.627077
        8            1          44.835944     17.290756     94.694446
                     2          20.984583     10.166067     45.524266
        9            1          49.715597     10.186532     90.595009
                     2          22.190987      9.217444     45.470983
        10           1         104.984047     19.843096    195.588622
                     2          43.853143     15.841895     97.608230
        11           1         125.780054     42.698836    237.746375
                     2          56.617000     22.615108    118.512361
        12           1         136.407384     42.236549    254.666726
                     2          62.994431     24.954156    136.213165
        13           1         204.743459     85.211820    320.977891
                     2          94.594572     42.930341    185.298264
        14           1         388.344389    150.643605    628.042444
                     2         181.206630     81.200588    319.013393
        15           1         648.909642    246.020122   1182.934505
                     2         306.948206    133.154036    608.741485
        16           1         742.553512    342.589391   1403.298503
                     2         367.197521    162.840228    779.455001
        17           1        1567.387803    674.485058   2234.324543
                     2         817.478668    330.592370   1723.574807
        18           1        2756.144235   1152.621818   4173.659617
                     2        1508.880046    568.759789   3009.980050
        19           1        3696.099029   1704.038134   5965.917801
                     2        2294.311719    907.434044   4574.024847
        20           1        5907.091535   2882.930262   8591.647026
                     2        4102.445784   1680.578398   7018.149752
        30           1       10374.060296   5567.590488  13467.752953
                     2        7765.041069   3564.983612  11806.353358
        31           1       14988.634029   9832.984977  19411.754435
                     2       12602.308993   6782.463249  18178.092953
        32           1       24965.013015  15964.984363  31154.906685
                     2       20847.696288   9967.871575  28980.545035
        235          1       35178.514352  17029.274623  49598.493142
                     2       32525.672784  12444.646220  47643.132488

# Explore age, sex, location, year ids for forecasted data

In [9]:
#age-group ids
acmr_all['age_group_id'].value_counts().sort_index()

2      3366000
3      3366000
4      3366000
5      3366000
6      3366000
7      3366000
8      3366000
9      3366000
10     3366000
11     3366000
12     3366000
13     3366000
14     3366000
15     3366000
16     3366000
17     3366000
18     3366000
19     3366000
20     3366000
30     3366000
31     3366000
32     3366000
235    3366000
Name: age_group_id, dtype: int64

In [8]:
acmr_all_loc['age_group_id'].unique()

array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  30,  31,  32, 235])

In [9]:
acmr_all_loc['sex_id'].unique()

array([1, 2])

In [10]:
#locations where acmr was forecasted for

acmr_location_ids = acmr_all['location_id'].unique()
acmr_location_ids_list = acmr_location_ids.tolist() #convert numpy.ndarray to a list
print(acmr_location_ids_list)
type(acmr_location_ids_list)
#acmr_location_ids is now a list

[354, 361, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521]


list

In [105]:
#GBD sex
sex = get_ids('sex')
sex

,sex_id,sex
0,1,Male
1,2,Female
2,3,Both
3,4,Unknown


In [11]:
#GBD location ids
all_locations = get_ids('location')
all_locations.head()

,location_id,location_name,location_description
0,1,Global,None
1,2,Developing,global
2,3,Developed,global
3,4,"Southeast Asia, East Asia, and Oceania",superregion
4,5,East Asia,region


In [12]:
all_locations[all_locations['location_name'].str.contains('Jiangsu')]

,location_id,location_name,location_description
466,506,Jiangsu,admin1


In [13]:
#get location names for locations in acmr forecast

acmr_location_names = all_locations[all_locations.location_id.isin(acmr_location_ids_list)].reset_index()
print(acmr_location_names)


    index  location_id                                     location_name  \
0     353          354  Hong Kong Special Administrative Region of China   
1     360          361      Macao Special Administrative Region of China   
2     451          491                                             Anhui   
3     452          492                                           Beijing   
4     453          493                                         Chongqing   
5     454          494                                            Fujian   
6     455          495                                             Gansu   
7     456          496                                         Guangdong   
8     457          497                                           Guangxi   
9     458          498                                           Guizhou   
10    459          499                                            Hainan   
11    460          500                                             Hebei   
12    461   

In [14]:
acmr_all['year_id'].unique()

array([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022,
       2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033,
       2034, 2035, 2036, 2037, 2038, 2039, 2040])

In [16]:
acmr_all['mr'].describe()

count    7.741800e+07
mean     6.012143e-02
std      1.325111e-01
min      5.388730e-12
25%      8.875841e-04
50%      5.444137e-03
75%      5.139766e-02
max      1.671621e+02
Name: mr, dtype: float64

# Subsetting to model locations, age_groups and year: acmr_alldraws

In [124]:
#model location and id lists

model_location_ids_list=[496,501,502,506,517]
model_years_list=list(range(2020, 2040))
model_agegroup_list=list(range(8,21)) + [30,31,32,235]
print(model_agegroup_list)
type(model_agegroup_list)

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 30, 31, 32, 235]


list

In [73]:
acmr_all.head()

,age_group_id,draw,location_id,sex_id,year_id,log_mr,mr,mr_100Kpy
0,2,0,354,1,1990,-1.852232,0.156887,15688.662762
1,2,0,354,1,1991,-1.935975,0.144283,14428.346328
2,2,0,354,1,1992,-1.989802,0.136723,13672.252803
3,2,0,354,1,1993,-2.198598,0.110959,11095.859481
4,2,0,354,1,1994,-2.292911,0.100972,10097.212142


First, get subset the df acmr_alldraws (which as log_mr and mr) with the right locations, age-groups and years that matches our sim

In [74]:
# SUBSET acmr for model_locations, age_groups, and years from acmr_all

acmr_alldraws=acmr_all[(acmr_all.age_group_id.isin(model_agegroup_list)) & (acmr_all.year_id.isin(model_years_list)) & (acmr_all.location_id.isin(model_location_ids_list))].reset_index(drop=True)
acmr_alldraws.head()


,age_group_id,draw,location_id,sex_id,year_id,log_mr,mr,mr_100Kpy
0,8,0,496,1,2020,-8.162255,0.000285,28.521862
1,8,0,496,1,2021,-8.128262,0.000295,29.508062
2,8,0,496,1,2022,-8.135679,0.000293,29.290011
3,8,0,496,1,2023,-8.136891,0.000293,29.254537
4,8,0,496,1,2024,-8.093703,0.000305,30.545664


# From-subsetted df (acmr_alldraws), obtain blended location acmr for year, age, sex

In [51]:
#1. Make a dictionary of location id and weight

province_weights = {517:0.18, 506:0.28, 496:0.22, 502:0.16, 501:0.16}

#the first position are the keys, the second position are the values
#list(province_weights.key())

In [75]:
#2. map weights to location in acmr_alldraws

acmr_alldraws['weight']=acmr_alldraws['location_id'].map(province_weights)
acmr_alldraws.head()

,age_group_id,draw,location_id,sex_id,year_id,log_mr,mr,mr_100Kpy,weight
0,8,0,496,1,2020,-8.162255,0.000285,28.521862,0.22
1,8,0,496,1,2021,-8.128262,0.000295,29.508062,0.22
2,8,0,496,1,2022,-8.135679,0.000293,29.290011,0.22
3,8,0,496,1,2023,-8.136891,0.000293,29.254537,0.22
4,8,0,496,1,2024,-8.093703,0.000305,30.545664,0.22


In [76]:
#3. obtain a weighted mr
acmr_alldraws['mr_weighted_100Kpy']=acmr_alldraws.mr_100Kpy*acmr_alldraws.weight
acmr_alldraws.head()

,age_group_id,draw,location_id,sex_id,year_id,log_mr,mr,mr_100Kpy,weight,mr_weighted_100Kpy
0,8,0,496,1,2020,-8.162255,0.000285,28.521862,0.22,6.274810
1,8,0,496,1,2021,-8.128262,0.000295,29.508062,0.22,6.491774
2,8,0,496,1,2022,-8.135679,0.000293,29.290011,0.22,6.443802
3,8,0,496,1,2023,-8.136891,0.000293,29.254537,0.22,6.435998
4,8,0,496,1,2024,-8.093703,0.000305,30.545664,0.22,6.720046


In [79]:
#4. sum over location by age_group_id, draw, sex_id and year_id
acmr_blended=acmr_alldraws.groupby(['year_id','age_group_id','sex_id','draw']).mr_weighted_100Kpy.sum().reset_index()
acmr_blended.head()

#680000 rows × 5 columns
#20years X 17 age groups, 2 sexes, 1000 draws

,year_id,age_group_id,sex_id,draw,mr_weighted_100Kpy
0,2020,8,1,0,26.599316
1,2020,8,1,1,30.477649
2,2020,8,1,10,28.565747
3,2020,8,1,100,25.160584
4,2020,8,1,101,25.086246


In [86]:
#check there are still 1000 draws

#acmr_blended['draw'].value_counts().sort_index()
acmr_blended['draw'].describe()

count     680000
unique      1000
top            7
freq         680
Name: draw, dtype: object

In [95]:
# summarize over draws and get 95%CIs, save to csv files

acmr_final=(acmr_blended.groupby(['year_id','age_group_id','sex_id'])
        .mr_weighted_100Kpy.describe(percentiles=[.025, 0.975])
        .filter(['mean', '2.5%', '97.5%']).reset_index())
acmr_final.head()

,year_id,age_group_id,sex_id,mean,2.5%,97.5%
0,2020,8,1,27.552543,23.949583,32.153055
1,2020,8,2,12.984441,10.729402,15.775554
2,2020,9,1,23.388177,17.326642,34.010046
3,2020,9,2,11.904669,9.848574,14.222933
4,2020,10,1,85.500117,68.793507,107.399174


In [109]:
age = {8:'15 to 19',
       9:'20 to 24',
       10:'25 to 29',
       11:'30 to 34',
       12:'35 to 39',
       13:'40 to 44',
       14:'45 to 49',
       15:'50 to 54',
       16:'55 to 59',
       17:'60 to 64',
       18:'65 to 69',
       19:'70 to 74',
       20:'75 to 79',
       30:'80 to 84',
       31:'85 to 89',
       32:'90 to 94',
       235:'95 plus'}

sex = {1:'male',2:'female'}

In [112]:
# age group values and sex values

acmr_final['age']=acmr_final['age_group_id'].map(age)
acmr_final['sex']=acmr_final['sex_id'].map(sex)
acmr_final['location']='blended'
acmr_final.head()

,year_id,age_group_id,age,sex_id,sex,mean,2.5%,97.5%,location
0,2020,8,15 to 19,1,male,27.552543,23.949583,32.153055,blended
1,2020,8,15 to 19,2,female,12.984441,10.729402,15.775554,blended
2,2020,9,20 to 24,1,male,23.388177,17.326642,34.010046,blended
3,2020,9,20 to 24,2,female,11.904669,9.848574,14.222933,blended
4,2020,10,25 to 29,1,male,85.500117,68.793507,107.399174,blended


# FINAL ACMR dataframe

In [113]:
columnsTitles = ['location','year_id', 'age_group_id', 'age', 'sex_id', 'sex', 'mean', '2.5%', '97.5%']

acmr_final = acmr_final.reindex(columns=columnsTitles)
acmr_final.head()

,location,year_id,age_group_id,age,sex_id,sex,mean,2.5%,97.5%
0,blended,2020,8,15 to 19,1,male,27.552543,23.949583,32.153055
1,blended,2020,8,15 to 19,2,female,12.984441,10.729402,15.775554
2,blended,2020,9,20 to 24,1,male,23.388177,17.326642,34.010046
3,blended,2020,9,20 to 24,2,female,11.904669,9.848574,14.222933
4,blended,2020,10,25 to 29,1,male,85.500117,68.793507,107.399174


In [123]:
%pwd
# %ls 
# acmr_final.to_csv('acmr_blendedlocation_forecast.csv', index=False)

'/ihme/costeffectiveness/vivarium_csu_cancer'

In [119]:
#export to csv

acmr_final.to_csv('acmr_blendedlocation_forecast.csv')

# CSMR for breast cancer (cause_id=429)

In [3]:
f_csmr = netCDF4.Dataset('429_ets_mortality.nc', 'r')
print(f_csmr)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): age_group_id(17), sex_id(2), location_id(33), year_id(51), draw(1000)
    variables(dimensions): int64 age_group_id(age_group_id), int64 sex_id(sex_id), int64 location_id(location_id), int64 year_id(year_id), int64 draw(draw), float64 mr(age_group_id,sex_id,location_id,year_id,draw)
    groups: 


In [5]:
csmr_mr=f_csmr.variables['mr']
print(csmr_mr)

<class 'netCDF4._netCDF4.Variable'>
float64 mr(age_group_id, sex_id, location_id, year_id, draw)
    _FillValue: nan
unlimited dimensions: 
current shape = (17, 2, 33, 51, 1000)
filling on


In [155]:
#1 . convert .nc file to pandas dataframe
ds_csmr = xr.open_dataset('429_ets_mortality.nc') #cause specific mortality file
csmr_all = ds_csmr.to_dataframe().reset_index()  
csmr_all.head()

,age_group_id,draw,location_id,sex_id,year_id,mr
0,8,0,354,1,1990,1.165002e-07
1,8,0,354,1,1991,1.126681e-07
2,8,0,354,1,1992,1.094101e-07
3,8,0,354,1,1993,1.056406e-07
4,8,0,354,1,1994,1.024418e-07


In [156]:
#2. convert mr to csmr 100Kpy

csmr_all['csmr']=csmr_all.mr*100000
csmr_all.head()


,age_group_id,draw,location_id,sex_id,year_id,mr,csmr
0,8,0,354,1,1990,1.165002e-07,0.011650
1,8,0,354,1,1991,1.126681e-07,0.011267
2,8,0,354,1,1992,1.094101e-07,0.010941
3,8,0,354,1,1993,1.056406e-07,0.010564
4,8,0,354,1,1994,1.024418e-07,0.010244


In [157]:
# 3. SUBSET csmr for model_locations, age_groups, and years from acmr_all

csmr_alldraws=csmr_all[(csmr_all.age_group_id.isin(model_agegroup_list)) & (csmr_all.year_id.isin(model_years_list)) & (csmr_all.location_id.isin(model_location_ids_list))].reset_index(drop=True)
csmr_alldraws.head()

# 3400000 rows × 7 columns
#5 provices x 20 years x 17 age groups x 2 sexes

,age_group_id,draw,location_id,sex_id,year_id,mr,csmr
0,8,0,496,1,2020,8.797803e-08,0.008798
1,8,0,496,1,2021,9.133636e-08,0.009134
2,8,0,496,1,2022,9.281358e-08,0.009281
3,8,0,496,1,2023,9.445730e-08,0.009446
4,8,0,496,1,2024,9.670490e-08,0.009670


In [158]:
#4 map location weights to location id

csmr_alldraws['weight']=csmr_alldraws['location_id'].map(province_weights)
csmr_alldraws.head()

,age_group_id,draw,location_id,sex_id,year_id,mr,csmr,weight
0,8,0,496,1,2020,8.797803e-08,0.008798,0.22
1,8,0,496,1,2021,9.133636e-08,0.009134,0.22
2,8,0,496,1,2022,9.281358e-08,0.009281,0.22
3,8,0,496,1,2023,9.445730e-08,0.009446,0.22
4,8,0,496,1,2024,9.670490e-08,0.009670,0.22


In [159]:
#5 multiply location weights by csmr to get weighted cmsr
csmr_alldraws['csmr_weighted']=csmr_alldraws['csmr']*csmr_alldraws['weight']
csmr_alldraws.head()

,age_group_id,draw,location_id,sex_id,year_id,mr,csmr,weight,csmr_weighted
0,8,0,496,1,2020,8.797803e-08,0.008798,0.22,0.001936
1,8,0,496,1,2021,9.133636e-08,0.009134,0.22,0.002009
2,8,0,496,1,2022,9.281358e-08,0.009281,0.22,0.002042
3,8,0,496,1,2023,9.445730e-08,0.009446,0.22,0.002078
4,8,0,496,1,2024,9.670490e-08,0.009670,0.22,0.002128


In [163]:
#6 sum over location by age_group_id, draw, sex_id and year_id

csmr_blended=csmr_alldraws.groupby(['year_id','age_group_id','sex_id','draw']).csmr_weighted.sum().reset_index()
csmr_blended.head()
#680000 rows × 5 columns


,year_id,age_group_id,sex_id,draw,csmr_weighted
0,2020,8,1,0,0.009352
1,2020,8,1,1,0.009014
2,2020,8,1,2,0.007802
3,2020,8,1,3,0.008454
4,2020,8,1,4,0.007637


In [164]:
#7 average over draws and get 95%CIs, save to csv files

csmr_final=(csmr_blended.groupby(['year_id','age_group_id','sex_id'])
        .csmr_weighted.describe(percentiles=[.025, 0.975])
        .filter(['mean', '2.5%', '97.5%']).reset_index())
csmr_final.head()

,year_id,age_group_id,sex_id,mean,2.5%,97.5%
0,2020,8,1,0.008418,0.007367,0.009627
1,2020,8,2,0.021894,0.017406,0.027173
2,2020,9,1,0.007272,0.006464,0.008300
3,2020,9,2,0.082557,0.066890,0.098687
4,2020,10,1,0.011853,0.009732,0.014849


In [166]:
#8 edit table columnes :age group values and sex values
columnsTitles = ['location','year_id', 'age_group_id', 'age', 'sex_id', 'sex', 'mean', '2.5%', '97.5%']

csmr_final['age']=csmr_final['age_group_id'].map(age)
csmr_final['sex']=csmr_final['sex_id'].map(sex)
csmr_final['location']='blended'

csmr_final = csmr_final.reindex(columns=columnsTitles)

csmr_final.head()

,location,year_id,age_group_id,age,sex_id,sex,mean,2.5%,97.5%
0,blended,2020,8,15 to 19,1,male,0.008418,0.007367,0.009627
1,blended,2020,8,15 to 19,2,female,0.021894,0.017406,0.027173
2,blended,2020,9,20 to 24,1,male,0.007272,0.006464,0.008300
3,blended,2020,9,20 to 24,2,female,0.082557,0.066890,0.098687
4,blended,2020,10,25 to 29,1,male,0.011853,0.009732,0.014849


In [168]:
#9 export to csv
csmr_final.to_csv('c429_csmr_blendedlocation_forecast.csv')
%pwd
%ls 

294_ets_mortality_beta_8_phi_89.nc*  429_ets_prevalence_beta_8_phi_89.nc*
429_ets_direct_incidence.nc*         acmr_blendedlocation_forecast.csv
429_ets_mortality.nc*                c429_csmr_blendedlocation_forecast.csv


# PREVALENCE for breast cancer (cause_id=429)

In [3]:
f_prev = netCDF4.Dataset('429_ets_prevalence_beta_8_phi_89.nc', 'r')
print(f_prev)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): age_group_id(17), sex_id(2), location_id(33), year_id(51), draw(1000)
    variables(dimensions): int64 age_group_id(age_group_id), int64 sex_id(sex_id), int64 location_id(location_id), int64 year_id(year_id), <class 'str'> draw(draw), float64 prevalence(age_group_id,sex_id,location_id,year_id,draw)
    groups: 


In [4]:
prev_prevalence=f_prev.variables['prevalence']
print(prev_prevalence)

<class 'netCDF4._netCDF4.Variable'>
float64 prevalence(age_group_id, sex_id, location_id, year_id, draw)
    _FillValue: nan
unlimited dimensions: 
current shape = (17, 2, 33, 51, 1000)
filling on


In [170]:
#1. convert .nc files to dataframes

ds_prev = xr.open_dataset('429_ets_prevalence_beta_8_phi_89.nc')
prev_all = ds_prev.to_dataframe().rename(columns={'prevalence':'log_prevalence'}).reset_index()  
prev_all.head()



,age_group_id,draw,location_id,sex_id,year_id,log_prevalence
0,8,0,354,1,1990,-12.314090
1,8,0,354,1,1991,-12.367723
2,8,0,354,1,1992,-12.414831
3,8,0,354,1,1993,-12.468393
4,8,0,354,1,1994,-12.489336


In [175]:
#2. convert prevalence by e transform, prevalence is now a proportion 

prev_all['prevalence']=np.exp(prev_all.log_prevalence)
prev_all.head()

,age_group_id,draw,location_id,sex_id,year_id,log_prevalence,prevalence
0,8,0,354,1,1990,-12.314090,0.000004
1,8,0,354,1,1991,-12.367723,0.000004
2,8,0,354,1,1992,-12.414831,0.000004
3,8,0,354,1,1993,-12.468393,0.000004
4,8,0,354,1,1994,-12.489336,0.000004


In [178]:
#3. subset 

prev_alldraws=prev_all[(prev_all.age_group_id.isin(model_agegroup_list)) & (prev_all.year_id.isin(model_years_list)) & (prev_all.location_id.isin(model_location_ids_list))].reset_index(drop=True)
prev_alldraws.head()

# 3400000 rows × 7 columns
#5 provices x 20 years x 17 age groups x 2 sexes


,age_group_id,draw,location_id,sex_id,year_id,log_prevalence,prevalence
0,8,0,496,1,2020,-12.256594,0.000005
1,8,0,496,1,2021,-12.250735,0.000005
2,8,0,496,1,2022,-12.241319,0.000005
3,8,0,496,1,2023,-12.241116,0.000005
4,8,0,496,1,2024,-12.242368,0.000005


In [179]:
#4 map location weights to location id

prev_alldraws['weight']=prev_alldraws['location_id'].map(province_weights)
prev_alldraws.head()

,age_group_id,draw,location_id,sex_id,year_id,log_prevalence,prevalence,weight
0,8,0,496,1,2020,-12.256594,0.000005,0.22
1,8,0,496,1,2021,-12.250735,0.000005,0.22
2,8,0,496,1,2022,-12.241319,0.000005,0.22
3,8,0,496,1,2023,-12.241116,0.000005,0.22
4,8,0,496,1,2024,-12.242368,0.000005,0.22


In [181]:
#5 multiply location weights by prev to get weighted prev
prev_alldraws['prev_weighted']=prev_alldraws['prevalence']*prev_alldraws['weight']
prev_alldraws.head()

,age_group_id,draw,location_id,sex_id,year_id,log_prevalence,prevalence,weight,prev_weighted
0,8,0,496,1,2020,-12.256594,0.000005,0.22,0.000001
1,8,0,496,1,2021,-12.250735,0.000005,0.22,0.000001
2,8,0,496,1,2022,-12.241319,0.000005,0.22,0.000001
3,8,0,496,1,2023,-12.241116,0.000005,0.22,0.000001
4,8,0,496,1,2024,-12.242368,0.000005,0.22,0.000001


In [184]:
#6 sum over location by age_group_id, draw, sex_id and year_id

prev_blended=prev_alldraws.groupby(['year_id','age_group_id','sex_id','draw']).prev_weighted.sum().reset_index()
prev_blended.head()
#680000 rows × 5 columns

,year_id,age_group_id,sex_id,draw,prev_weighted
0,2020,8,1,0,0.000005
1,2020,8,1,1,0.000005
2,2020,8,1,10,0.000005
3,2020,8,1,100,0.000005
4,2020,8,1,101,0.000005


In [185]:
#7 average over draws and get 95%CIs, save to csv files

prev_final=(prev_blended.groupby(['year_id','age_group_id','sex_id'])
        .prev_weighted.describe(percentiles=[.025, 0.975])
        .filter(['mean', '2.5%', '97.5%']).reset_index())
prev_final.head()

,year_id,age_group_id,sex_id,mean,2.5%,97.5%
0,2020,8,1,0.000005,0.000005,0.000005
1,2020,8,2,0.000026,0.000024,0.000028
2,2020,9,1,0.000005,0.000005,0.000006
3,2020,9,2,0.000122,0.000114,0.000132
4,2020,10,1,0.000009,0.000009,0.000010


In [186]:
#8 edit table columnes :age group values and sex values
columnsTitles = ['location','year_id', 'age_group_id', 'age', 'sex_id', 'sex', 'mean', '2.5%', '97.5%']

prev_final['age']=prev_final['age_group_id'].map(age)
prev_final['sex']=prev_final['sex_id'].map(sex)
prev_final['location']='blended'

prev_final = prev_final.reindex(columns=columnsTitles)

prev_final.head()

,location,year_id,age_group_id,age,sex_id,sex,mean,2.5%,97.5%
0,blended,2020,8,15 to 19,1,male,0.000005,0.000005,0.000005
1,blended,2020,8,15 to 19,2,female,0.000026,0.000024,0.000028
2,blended,2020,9,20 to 24,1,male,0.000005,0.000005,0.000006
3,blended,2020,9,20 to 24,2,female,0.000122,0.000114,0.000132
4,blended,2020,10,25 to 29,1,male,0.000009,0.000009,0.000010


In [188]:
#9 
prev_final.to_csv('c429_prev_blendedlocation_forecast.csv')
%pwd
%ls 

294_ets_mortality_beta_8_phi_89.nc*   acmr_blendedlocation_forecast.csv
429_ets_direct_incidence.nc*          c429_csmr_blendedlocation_forecast.csv
429_ets_mortality.nc*                 c429_prev_blendedlocation_forecast.csv
429_ets_prevalence_beta_8_phi_89.nc*


# INCIDENCE for breast cancer (cause_id=429)

In [8]:
#1. convert .nc files to dataframes

ds_inc = xr.open_dataset('429_ets_direct_incidence.nc')
incidence_all = ds_inc.to_dataframe().rename(columns={'incidence':'incidence_perpy'}).reset_index()  
incidence_all.head()

,age_group_id,draw,location_id,sex_id,year_id,incidence_perpy
0,8,0,354,1,1990,5.308013e-07
1,8,0,354,1,1991,5.128159e-07
2,8,0,354,1,1992,4.769136e-07
3,8,0,354,1,1993,4.210661e-07
4,8,0,354,1,1994,4.386884e-07


In [9]:
#2. convert incidence_perpy to incidence (per 100K py)

incidence_all['incidence']=incidence_all.incidence_perpy*100000
incidence_all.head()

,age_group_id,draw,location_id,sex_id,year_id,incidence_perpy,incidence
0,8,0,354,1,1990,5.308013e-07,0.053080
1,8,0,354,1,1991,5.128159e-07,0.051282
2,8,0,354,1,1992,4.769136e-07,0.047691
3,8,0,354,1,1993,4.210661e-07,0.042107
4,8,0,354,1,1994,4.386884e-07,0.043869


In [10]:
#3. subset 

incidence_alldraws=incidence_all[(incidence_all.age_group_id.isin(model_agegroup_list)) & (incidence_all.year_id.isin(model_years_list)) & (incidence_all.location_id.isin(model_location_ids_list))].reset_index(drop=True)
incidence_alldraws.head()

# 3400000 rows × 7 columns
#5 provices x 20 years x 17 age groups x 2 sexes


,age_group_id,draw,location_id,sex_id,year_id,incidence_perpy,incidence
0,8,0,496,1,2020,5.665471e-07,0.056655
1,8,0,496,1,2021,5.732215e-07,0.057322
2,8,0,496,1,2022,5.773528e-07,0.057735
3,8,0,496,1,2023,5.568807e-07,0.055688
4,8,0,496,1,2024,5.570711e-07,0.055707


In [13]:
#4 map location weights to location id

incidence_alldraws['weight']=incidence_alldraws['location_id'].map(province_weights)
incidence_alldraws.head()

,age_group_id,draw,location_id,sex_id,year_id,incidence_perpy,incidence,weight
0,8,0,496,1,2020,5.665471e-07,0.056655,0.22
1,8,0,496,1,2021,5.732215e-07,0.057322,0.22
2,8,0,496,1,2022,5.773528e-07,0.057735,0.22
3,8,0,496,1,2023,5.568807e-07,0.055688,0.22
4,8,0,496,1,2024,5.570711e-07,0.055707,0.22


In [15]:
#5 multiply location weights by incidence to get weighted incidence
incidence_alldraws['incidence_weighted']=incidence_alldraws['incidence']*incidence_alldraws['weight']
incidence_alldraws.head()

,age_group_id,draw,location_id,sex_id,year_id,incidence_perpy,incidence,weight,incidence_weighted
0,8,0,496,1,2020,5.665471e-07,0.056655,0.22,0.012464
1,8,0,496,1,2021,5.732215e-07,0.057322,0.22,0.012611
2,8,0,496,1,2022,5.773528e-07,0.057735,0.22,0.012702
3,8,0,496,1,2023,5.568807e-07,0.055688,0.22,0.012251
4,8,0,496,1,2024,5.570711e-07,0.055707,0.22,0.012256


In [16]:
#6 sum over location by age_group_id, draw, sex_id and year_id

incidence_blended=incidence_alldraws.groupby(['year_id','age_group_id','sex_id','draw']).incidence_weighted.sum().reset_index()
incidence_blended.head()
#680000 rows × 5 columns

,year_id,age_group_id,sex_id,draw,incidence_weighted
0,2020,8,1,0,0.053296
1,2020,8,1,1,0.056567
2,2020,8,1,2,0.048804
3,2020,8,1,3,0.054708
4,2020,8,1,4,0.055786


In [17]:
#7 average over draws and get 95%CIs, save to csv files

incidence_final=(incidence_blended.groupby(['year_id','age_group_id','sex_id'])
        .incidence_weighted.describe(percentiles=[.025, 0.975])
        .filter(['mean', '2.5%', '97.5%']).reset_index())
incidence_final.head()

,year_id,age_group_id,sex_id,mean,2.5%,97.5%
0,2020,8,1,0.055150,0.045545,0.068971
1,2020,8,2,0.278463,0.231600,0.334534
2,2020,9,1,0.054776,0.047745,0.062537
3,2020,9,2,1.226677,1.047347,1.426381
4,2020,10,1,0.100310,0.085657,0.116574


In [20]:
#8 edit table columnes :age group values and sex values
columnsTitles = ['location','year_id', 'age_group_id', 'age', 'sex_id', 'sex', 'mean', '2.5%', '97.5%']

incidence_final['age']=incidence_final['age_group_id'].map(age)
incidence_final['sex']=incidence_final['sex_id'].map(sex)
incidence_final['location']='blended'

incidence_final = incidence_final.reindex(columns=columnsTitles)

incidence_final.head()

,location,year_id,age_group_id,age,sex_id,sex,mean,2.5%,97.5%
0,blended,2020,8,15 to 19,1,male,0.055150,0.045545,0.068971
1,blended,2020,8,15 to 19,2,female,0.278463,0.231600,0.334534
2,blended,2020,9,20 to 24,1,male,0.054776,0.047745,0.062537
3,blended,2020,9,20 to 24,2,female,1.226677,1.047347,1.426381
4,blended,2020,10,25 to 29,1,male,0.100310,0.085657,0.116574


In [21]:
#9 
incidence_final.to_csv('c429_incidence_blendedlocation_forecast.csv')
%pwd
%ls 

294_ets_mortality_beta_8_phi_89.nc*
429_ets_direct_incidence.nc*
429_ets_mortality.nc*
429_ets_prevalence_beta_8_phi_89.nc*
acmr_blendedlocation_forecast.csv
c429_csmr_blendedlocation_forecast.csv
c429_incidence_blendedlocation_forecast.csv
c429_prev_blendedlocation_forecast.csv
